## This notebook is used to retrieve and format the Charging Station level data. 



First, the notebook calls the alternative fuel center's api to get the data related to electric charging stations they have cataloged.

Then, for each of those electric charging stations, we leverage the FCC's api and, using the latitude and longitude of the station, can get the FIPS code.

*Note, the act of running this file results in multiple data files being generated. Moreover, the FCC api call is not batch. It will take about 4 hours to get FIPS code from api call

Calling api to get Stations

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
import requests
import urllib
from tqdm import tqdm

In [15]:
url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?api_key=NO5UMIkC119SbwWzeK5djfAwj5fF8RvrSL807Mim&fuel_type=ELEC'

In [16]:
response = requests.get(url)

In [17]:
df = pd.json_normalize(response.json()['fuel_stations'])
df

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,open_date,owner_type_code,status_code,restricted_access,maximum_vehicle_class,station_name,station_phone,updated_at,facility_type,geocode_status,latitude,longitude,city,intersection_directions,plus4,state,street_address,zip,country,bd_blends,cng_dispenser_num,cng_fill_type_code,cng_psi,cng_renewable_source,cng_total_compression,cng_total_storage,cng_vehicle_class,cng_has_rng,e85_blender_pump,e85_other_ethanol_blends,ev_connector_types,ev_dc_fast_num,ev_level1_evse_num,ev_level2_evse_num,ev_network,ev_network_web,ev_other_evse,ev_pricing,ev_renewable_source,hy_is_retail,hy_pressures,hy_standards,hy_status_link,lng_renewable_source,lng_vehicle_class,lng_has_rng,lpg_primary,lpg_nozzle_types,ng_fill_type_code,ng_psi,ng_vehicle_class,rd_blends,rd_blends_fr,rd_blended_with_biodiesel,rd_max_biodiesel_level,nps_unit_name,access_days_time_fr,intersection_directions_fr,bd_blends_fr,groups_with_access_code_fr,ev_pricing_fr,ev_network_ids.station,ev_network_ids.posts,federal_agency.id,federal_agency.code,federal_agency.name
0,private,Fleet use only,None,None,2023-01-10,None,ELEC,Private,1517,1999-10-15,LG,E,None,None,LADWP - Truesdale Center,None,2023-02-15T22:45:41Z,UTILITY,GPS,34.248319,-118.387971,Sun Valley,None,None,CA,11797 Truesdale St,91352,US,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772, J1772COMBO]",3.0,NaN,39.0,Non-Networked,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Privé,None,NaN,NaN,NaN,NaN,NaN
1,private,None,None,None,2023-01-10,None,ELEC,Private,1519,2020-02-28,LG,E,None,None,LADWP - West LA District Office,None,2023-02-15T22:45:41Z,UTILITY,200-8,34.052542,-118.448504,Los Angeles,None,None,CA,1394 S Sepulveda Blvd,90024,US,None,None,None,None,None,None,None,None,None,None,None,[J1772],NaN,NaN,4.0,Non-Networked,None,None,Free,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Privé,None,NaN,NaN,NaN,NaN,NaN
2,public,5:30am-9pm; pay lot,None,None,2023-01-10,None,ELEC,Public,1523,1995-08-30,P,E,False,LD,Los Angeles Convention Center,213-741-1151,2023-02-14T15:54:11Z,PARKING_GARAGE,GPS,34.040539,-118.271387,Los Angeles,West hall and South hall,None,CA,1201 S Figueroa St,90015,US,None,None,None,None,None,None,None,None,None,None,None,[J1772],NaN,NaN,7.0,Non-Networked,None,None,Free; parking fee,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Public,None,NaN,NaN,NaN,NaN,NaN
3,private,For fleet and employee use only,None,None,2023-01-10,None,ELEC,Private,1525,1999-10-15,LG,E,None,None,LADWP - John Ferraro Building,None,2023-02-15T22:45:41Z,UTILITY,GPS,34.059133,-118.248589,Los Angeles,Across Hope,None,CA,111 N Hope St,90012,US,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772, J1772COMBO]",2.0,NaN,311.0,Non-Networked,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Privé,None,NaN,NaN,NaN,NaN,NaN
4,private,Fleet use only,None,None,2023-01-10,None,ELEC,Private,1531,2018-05-01,LG,E,None,None,LADWP - Haynes Power Plant,None,2023-02-15T22:45:41Z,UTILITY,GPS,33.759802,-118.096665,Long Beach,None,None,CA,6801 E 2nd St,90803,US,None,None,None,None,None,None,None,None,None,None,None,"[CHADEMO, J1772, J1772COMBO]",1.0,NaN,19.0,Non-Networked,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Privé,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62094,public,2

In [19]:
df.to_csv('Charging_station_by_county.csv')

pinging API to see FIPS code

In [42]:
df = pd.read_csv('Charging_station_by_county.csv')

C:\Users\zapate\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (27,46,65,74,75) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
df1 = df[['open_date', 'latitude', 'longitude', 'state', 'ev_network', 'status_code']]
df1

,open_date,latitude,longitude,state,ev_network,status_code
0,1999-10-15,34.248319,-118.387971,CA,Non-Networked,E
1,2020-02-28,34.052542,-118.448504,CA,Non-Networked,E
2,1995-08-30,34.040539,-118.271387,CA,Non-Networked,E
3,1999-10-15,34.059133,-118.248589,CA,Non-Networked,E
4,2018-05-01,33.759802,-118.096665,CA,Non-Networked,E
...,...,...,...,...,...,...
62094,2023-07-20,47.044688,-122.906556,WA,SHELL_RECHARGE,E
62095,2023-07-20,37.675809,-97.393545,KS,EV Connect,E
62096,2023-07-20,39.627990,-78.018151,MD,EV Connect,E
62097,2023-07-20,34.200030,-118.614542,CA,FLO,E


In [26]:
df1.iloc[1-1][['latitude', 'longitude']][0]

34.2483191527193

In [27]:
fips_list = []

#fips collection
for i in tqdm(range(1, len(df1) + 1)):
    lat = df1.iloc[i-1][['latitude', 'longitude']][0]
    long = df1.iloc[i-1][['latitude', 'longitude']][1]
    params = urllib.parse.urlencode({'latitude': lat, 'longitude':long, 'censusYear': 2010, 'format':'json'})

    url = 'https://geo.fcc.gov/api/census/block/find?' + params
    response = requests.get(url)

    data = response.json()
    fips_list.append(data['County']['FIPS'])
    
    if i%1000 == 0 or i == len(df1):
        path = r'C:/Users/zapate/Desktop/YEAR 2/Charging Stations/fips_code_2010_' + str(i) + '.csv'
        with open(path, 'w') as fp:
            for item in fips_list:
                fp.write("%s\n" % item)
            print(i)
        fips_list = []

'''        
fips_list = []

#first 610000
for i in tqdm(1,range(len(df1) + 1)):
    lat = df1.iloc[i][['latitude', 'longitude']][0]
    long = df1.iloc[i][['latitude', 'longitude']][1]
    params = urllib.parse.urlencode({'latitude': lat, 'longitude':long, 'censusYear': 2010, 'format':'json'})

    url = 'https://geo.fcc.gov/api/census/block/find?' + params
    response = requests.get(url)

    data = response.json()
    fips_list.append(data['County']['FIPS'])
    
    if i%1000 == 0:
        path = r'C:/Users/zapate/Desktop/YEAR 2/Charging Stations/fips_code_2010_' + str(i) + '.csv'
        with open(path, 'w') as fp:
            for item in fips_list:
                fp.write("%s\n" % item)
            print(i)
        fips_list = []
'''

  2%|▏         | 1001/62099 [03:47<3:16:20,  5.19it/s]

1000


  3%|▎         | 2001/62099 [07:27<2:53:37,  5.77it/s]

2000


  5%|▍         | 3001/62099 [11:18<2:52:01,  5.73it/s]

...

100%|█████████▉| 62001/62099 [3:55:42<00:18,  5.38it/s]

62000


100%|██████████| 62099/62099 [3:56:03<00:00,  4.38it/s]

62099


'        \nfips_list = []\n\n#first 610000\nfor i in tqdm(1,range(len(df1) + 1)):\n    lat = df1.iloc[i][[\'latitude\', \'longitude\']][0]\n    long = df1.iloc[i][[\'latitude\', \'longitude\']][1]\n    params = urllib.parse.urlencode({\'latitude\': lat, \'longitude\':long, \'censusYear\': 2010, \'format\':\'json\'})\n\n    url = \'https://geo.fcc.gov/api/census/block/find?\' + params\n    response = requests.get(url)\n\n    data = response.json()\n    fips_list.append(data[\'County\'][\'FIPS\'])\n    \n    if i%1000 == 0:\n        path = r\'C:/Users/zapate/Desktop/YEAR 2/Charging Stations/fips_code_2010_\' + str(i) + \'.csv\'\n        with open(path, \'w\') as fp:\n            for item in fips_list:\n                fp.write("%s\n" % item)\n            print(i)\n        fips_list = []\n'

In [28]:
'''
fips_list = []
#after 61000
for i in tqdm(range(61000, len(df1))):
    lat = df1.iloc[i][['latitude', 'longitude']][0]
    long = df1.iloc[i][['latitude', 'longitude']][1]
    params = urllib.parse.urlencode({'latitude': lat, 'longitude':long, 'censusYear': 2010, 'format':'json'})

    url = 'https://geo.fcc.gov/api/census/block/find?' + params
    response = requests.get(url)

    data = response.json()
    fips_list.append(data['County']['FIPS'])
    
path = r'C:/Users/zapate/Desktop/YEAR 2/Charging Stations/fips_code_2010_' + str(62) + '.csv'
with open(path, 'w') as fp:
    for item in fips_list:
        fp.write("%s\n" % item)
    print(i)
    

fips_list = []       

for i in tqdm(range(61000, len(df1))):
    lat = df1.iloc[i][['latitude', 'longitude']][0]
    long = df1.iloc[i][['latitude', 'longitude']][1]
    params = urllib.parse.urlencode({'latitude': lat, 'longitude':long, 'censusYear': 2020, 'format':'json'})

    url = 'https://geo.fcc.gov/api/census/block/find?' + params
    response = requests.get(url)

    data = response.json()
    fips_list.append(data['County']['FIPS'])
    
path = r'C:/Users/zapate/Desktop/YEAR 2/Charging Stations/fips_code_' + str(62) + '.csv'
with open(path, 'w') as fp:
    for item in fips_list:
        fp.write("%s\n" % item)
    print(i)
'''


'\nfips_list = []\n#after 61000\nfor i in tqdm(range(61000, len(df1))):\n    lat = df1.iloc[i][[\'latitude\', \'longitude\']][0]\n    long = df1.iloc[i][[\'latitude\', \'longitude\']][1]\n    params = urllib.parse.urlencode({\'latitude\': lat, \'longitude\':long, \'censusYear\': 2010, \'format\':\'json\'})\n\n    url = \'https://geo.fcc.gov/api/census/block/find?\' + params\n    response = requests.get(url)\n\n    data = response.json()\n    fips_list.append(data[\'County\'][\'FIPS\'])\n    \npath = r\'C:/Users/zapate/Desktop/YEAR 2/Charging Stations/fips_code_2010_\' + str(62) + \'.csv\'\nwith open(path, \'w\') as fp:\n    for item in fips_list:\n        fp.write("%s\n" % item)\n    print(i)\n    \n\nfips_list = []       \n\nfor i in tqdm(range(61000, len(df1))):\n    lat = df1.iloc[i][[\'latitude\', \'longitude\']][0]\n    long = df1.iloc[i][[\'latitude\', \'longitude\']][1]\n    params = urllib.parse.urlencode({\'latitude\': lat, \'longitude\':long, \'censusYear\': 2020, \'format\':

In [29]:
len(df1)

62099

2020 fips codes read csv

In [44]:
'''
fips_df = pd.read_csv('fips_code1000.csv', header= None)
fips_df.columns = ['FIPS_2020']

for i in range(2,63):
    stg_df = pd.read_csv('fips_code' + str(i) + '000.csv', header = None)
    stg_df.columns = ['FIPS_2020']
    fips_df = pd.concat([fips_df, stg_df], ignore_index = True)
    
df1['FIPS_2020'] = fips_df['FIPS_2020']
'''    
fips_df = pd.read_csv('fips_code_2010_1000.csv', header= None)
fips_df.columns = ['FIPS_2010']

for i in range(2,63):
    stg_df = pd.read_csv('fips_code_2010_' + str(i) + '000.csv', header = None)
    stg_df.columns = ['FIPS_2010']
    fips_df = pd.concat([fips_df, stg_df], ignore_index = True)
    
stg_df = pd.read_csv('fips_code_2010_62099.csv', header = None)
stg_df.columns = ['FIPS_2010']
fips_df = pd.concat([fips_df, stg_df], ignore_index = True)
    
df1['FIPS_2010'] = fips_df['FIPS_2010']
  

C:\Users\zapate\AppData\Local\Temp\ipykernel_44912\3888010483.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['FIPS_2010'] = fips_df['FIPS_2010']


In [45]:
fips_df

,FIPS_2010
0,6037
1,6037
2,6037
3,6037
4,6037
...,...
62094,53067
62095,20173
62096,24043
62097,06037


In [46]:
#df1['FIPS_2020'] = pd.to_numeric(df1['FIPS_2020'],errors = 'coerce', downcast = 'integer')
df1['FIPS_2010'] = pd.to_numeric(df1['FIPS_2010'],errors = 'coerce', downcast = 'integer')


C:\Users\zapate\AppData\Local\Temp\ipykernel_44912\2683442280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['FIPS_2010'] = pd.to_numeric(df1['FIPS_2010'],errors = 'coerce', downcast = 'integer')


In [41]:
df1

,open_date,latitude,longitude,state,ev_network,FIPS_2010
0,1999-10-15,34.248319,-118.387971,CA,Non-Networked,6037.0
1,2020-02-28,34.052542,-118.448504,CA,Non-Networked,6037.0
2,1995-08-30,34.040539,-118.271387,CA,Non-Networked,6037.0
3,1999-10-15,34.059133,-118.248589,CA,Non-Networked,6037.0
4,2018-05-01,33.759802,-118.096665,CA,Non-Networked,6037.0
...,...,...,...,...,...,...
62094,2023-07-20,47.044688,-122.906556,WA,SHELL_RECHARGE,53067.0
62095,2023-07-20,37.675809,-97.393545,KS,EV Connect,20173.0
62096,2023-07-20,39.627990,-78.018151,MD,EV Connect,24043.0
62097,2023-07-20,34.200030,-118.614542,CA,FLO,6037.0


In [48]:
df1.to_csv('Charging_station_by_county_calculations_final.csv')

In [38]:
fips_df.to_csv('Final_charging_station_FIPS_Codes.csv')